## <center> Trainner Script 
we are using our custom TrafficSignNet 

In [2]:
from traffic_sign_net import TrafficSignNet # custom Sequential model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report

We will use scikit-image for preprocessing our dataset in lieu of OpenCV as scikit-image provides some additional preprocessing algorithms that OpenCV does not 

In [5]:
from skimage import transform
from skimage import exposure
from skimage import io

In [8]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import random
import os

initialize the list of data and labels

In [10]:
def load_split(basePath, csvPath):
    data = []
    labels = []
    
    #The format of the data is: Width, Height, X1, Y1, X2, Y2, ClassID, Image Path
    rows = open(csvPath).read().strip().split('\n')[1:] # skipping the first header row
    random.shuffle(rows)
    
    #progressBar
    pbar = tqdm(total=len(rows), desc="loaded:")
    for (i, row) in enumerate(rows):
        (label, imagePath) = row.strip().split(",")[-2:]
        
        imagePath = os.path.sep.join([basePath, imagePath])
        image = io.imread(imagePath)
        
        #images by applying CLAHE
        image = transform.resize(image, (32, 32)) #resize
        image = exposure.equalize_adapthist(image, clip_limit=0.1)
        
        data.append(image)
        labels.append(int(label))
        
        pbar.update(1) #progessBAr
    pbar.close()
        
    
    #convert to numpy
    data = np.array(data)
    labels = np.array(labels)
    
    return (data, labels)

In [12]:
# epochs to train for, base learning rate and batch size
EPOCHS = 30
INIT_LR = 1e-3 # 0.001
BS = 64

# load the label names
labelNames = open("signnames.csv").read().strip().split("\n")[1:] #except header
labelNames = [l.split(",")[1] for l in labelNames]
print("numOf Sign Can  be detect ", len(labelNames))

numOf Sign Can  be detect  43


In [13]:
trainPath = r'dataset/Train.csv'
textPath = r'dataset/Test.csv'

print("🔃: loading dataset")
(trainX, trainY) =  load_split("dataset", trainPath)
(testX, testY) = load_split("dataset", textPath)

🔃: loading dataset


loaded::   0%|          | 0/39209 [00:00<?, ?it/s]

loaded::   0%|          | 0/12630 [00:00<?, ?it/s]

In [14]:
#scale range [0, 1]
trainX = trainX.astype("float32") /255.0
testX = testX.astype("float32") /255.0

In [16]:
#oneHot encode the train & testing labels
numOfLabel = len(np.unique(trainY))
trainY = to_categorical(trainY, numOfLabel)
testY = to_categorical(testY, numOfLabel)

we have significantly more images for some classes than others

In [18]:
# account for skew in the labeled data 
classTotals = trainY.sum(axis=0)
classWeight = classTotals.max() / classTotals

#### image generator for data augmentation 
 we’re not applying horizontal or vertical flips here as 
traffic signs in the wild will not be flipped.

In [20]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")